In [1]:
# variables = ['datadate', 'ticker', 'agr', 'bm', 'mom12m', 'mve', 'operprof', 'roeq', 'absacc', 'acc', 'aeavol', 'age', 'baspread', 'BETA', 'bm_ia', 
#              'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chfeps', 'chinv', 'chmom', 'chnanalyst', 
#              'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'disp', 'divi', 'divo', 'dy', 'ear', 'egr', 'ep', 'fgr5yr', 
#              'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'idiovol', 'ill', 'indmom', 'invest', 'IPO', 'lev', 'mom1m', 'mom36m', 'ms', 
#              'mve_ia', 'nanalyst', 'nincr', 'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchsale_pchinvt', 
#              'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'pricedelay', 'ps', 'rd', 'rd_mve', 'rd_sale', 'realestate', 
#              'retvol', 'roaq', 'roavol', 'roic', 'rsup', 'salecash', 'saleinv', 'salerec', 'secured', 'securedind', 'sfe', 'sgr', 'sin', 
#              'sp', 'std_dolvol', 'std_turn', 'stdcf', 'sue', 'tang', 'tb', 'turn', 'zerotrade']
variables = ['datadate', 'ticker', 'bm', 'mom12m', 'mve']

In [2]:
#from agentics import AG

In [3]:
#green_cleaned = AG.from_csv('../green cleaned.csv', max_rows = 10)

In [4]:
#green_cleaned[0]

In [5]:
# for x in green_cleaned:
#     print(x.bm)

In [6]:
# #green_cleaned = green_cleaned.add_attribute('recommendation', description='Buy, hold, or sell recommendation for given ticker')
# from pydantic import BaseModel

# class Recommendation(BaseModel):
#     recommendation: str | None = None

# target = AG(atype = Recommendation, instructions='Determine a buy, hold, or sell for given ticker.')

In [7]:
# output = await (target << green_cleaned)
# output.pretty_print()

In [8]:
#green_cleaned = await green_cleaned.self_transduction(['bm', 'mve', 'mom12m', 'ticker'], ['recommendation'])

In [2]:
from dotenv import find_dotenv, load_dotenv
from getpass import getpass
import os

#load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Enter your GEMINI_API_KEY: ········


In [3]:
import sqlite3
import pandas as pd
from crewai.tools import tool
from crewai import Agent, Task, Crew, Process
from agentics import AG
import time
from functools import wraps
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

# ============================================
# Rate Limiting Setup
# ============================================
class RateLimiter:
    def __init__(self, calls_per_minute=5):
        self.calls_per_minute = calls_per_minute
        self.interval = 60.0 / calls_per_minute
        self.last_call = 0
        self.call_count = 0
        
    def wait_if_needed(self):
        """Wait if necessary to respect rate limit."""
        current_time = time.time()
        time_since_last = current_time - self.last_call
        
        if time_since_last < self.interval:
            sleep_time = self.interval - time_since_last
            print(f"[Rate Limit] Waiting {sleep_time:.2f}s (Call #{self.call_count + 1})")
            time.sleep(sleep_time)
        
        self.last_call = time.time()
        self.call_count += 1

rate_limiter = RateLimiter(calls_per_minute=5)

def rate_limit(func):
    """Decorator with retry logic for rate limiting."""
    @wraps(func)
    @retry(
        retry=retry_if_exception_type((Exception,)),
        wait=wait_exponential(multiplier=1, min=4, max=60),
        stop=stop_after_attempt(5)
    )
    def wrapper(*args, **kwargs):
        rate_limiter.wait_if_needed()
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"[Error] {func.__name__} failed: {str(e)}")
            raise
    return wrapper

# ============================================
# Database Setup
# ============================================
def setup_firm_database(csv_path, db_path='firm_data.db', selected_columns=None):
    """Load your CSV into SQLite database (run this once)."""
    print(f"Loading CSV from {csv_path}...")
    
    sample_df = pd.read_csv(csv_path, nrows=5)
    
    if selected_columns is None:
        print(f"\nAvailable columns ({len(sample_df.columns)} total):")
        for i, col in enumerate(sample_df.columns, 1):
            print(f"  {i}. {col}")
        
        response = input("\nPress Enter to use ALL columns, or type 'stop' to exit: ").strip()
        if response.lower() == 'stop':
            print("Exiting.")
            return
        selected_columns = sample_df.columns.tolist()
    else:
        missing = [col for col in selected_columns if col not in sample_df.columns]
        if missing:
            print(f"\n❌ Error: These columns don't exist: {missing}")
            return
    
    clean_columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_')
                    .replace('(', '').replace(')', '').replace('/', '_') 
                    for col in selected_columns]
    
    import os
    if os.path.exists(db_path):
        os.remove(db_path)
    
    chunk_size = 50000
    conn = sqlite3.connect(db_path)
    
    first_chunk = True
    chunk_num = 0
    total_rows = 0
    
    print("\nLoading data...")
    for chunk in pd.read_csv(csv_path, chunksize=chunk_size, usecols=selected_columns):
        chunk.columns = clean_columns
        chunk_num += 1
        total_rows += len(chunk)
        
        if first_chunk:
            chunk.to_sql('firms', conn, if_exists='replace', index=False)
            first_chunk = False
        else:
            chunk.to_sql('firms', conn, if_exists='append', index=False)
        
        print(f"  Chunk {chunk_num}: {total_rows:,} rows")
    
    cursor = conn.cursor()
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_ticker ON firms(ticker)")
    cursor.execute("CREATE INDEX IF NOT EXISTS idx_date ON firms(datadate)")
    conn.commit()
    conn.close()
    
    print(f"\n✓ Database created: {db_path} ({total_rows:,} rows)")
    return clean_columns

# ============================================
# Database Query Tools
# ============================================
@tool("query_firm_database")
@rate_limit
def query_firm_database(sql_query: str) -> str:
    """
    Execute SQL queries on the S&P 500 firm characteristics database.
    
    Available table: 'firms' (contains all firm characteristic data)
    
    IMPORTANT DATA NOTES:
    - 'mve' column = log(market value of equity), i.e., log size
    - ALL features are standardized: mean = 0, standard deviation = 1
    - Higher values = more extreme positive, lower values = more extreme negative
    
    Example queries:
    - SELECT * FROM firms WHERE datadate = '2019-12-31'
    - SELECT AVG(bm), AVG(mom12m), AVG(mve) FROM firms WHERE datadate = '2019-12-31'
    - SELECT ticker, bm, mom12m FROM firms WHERE datadate = '2019-12-31' AND bm < -1 ORDER BY bm LIMIT 20
    
    Args:
        sql_query: A valid SQL SELECT query
    
    Returns:
        Query results as a formatted string
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        df = pd.read_sql_query(sql_query, conn)
        conn.close()
        
        if df.empty:
            return "Query returned no results."
        
        result = f"Query returned {len(df)} rows:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error executing query: {str(e)}"

@tool("get_database_schema")
@rate_limit
def get_database_schema() -> str:
    """
    Get the schema of the firms database including all column names and types.
    
    CRITICAL DATA INFORMATION:
    - 'mve' = log(market value of equity), represents log firm size
    - ALL features are standardized to mean=0, std=1
    - Values represent standard deviations from mean (z-scores)
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        cursor = conn.cursor()
        
        cursor.execute("PRAGMA table_info(firms)")
        columns = cursor.fetchall()
        
        cursor.execute("SELECT COUNT(*) FROM firms")
        row_count = cursor.fetchone()[0]
        
        cursor.execute("SELECT MIN(datadate), MAX(datadate) FROM firms")
        date_range = cursor.fetchone()
        
        conn.close()
        
        schema = f"""Database: firm_data.db
Table: firms
Total rows: {row_count:,}
Date range: {date_range[0]} to {date_range[1]}

DATA STANDARDIZATION:
- 'mve' column = log(market value of equity) - represents log firm size
- 'bm' column = log(book to market) - represents whether a firm is overvalued or undervalued
- 'mom12m' column = 12 month momentum
- ALL features are standardized: mean = 0, standard deviation = 1
- Interpret values as z-scores (standard deviations from mean)
- Example: A value of 2.0 means 2 standard deviations above average
- Example: A value of -1.5 means 1.5 standard deviations below average

Columns ({len(columns)} total):
"""
        for col in columns:
            schema += f"  - {col[1]} ({col[2]})\n"
        
        return schema
    except Exception as e:
        return f"Error retrieving schema: {str(e)}"

@tool("get_distribution_stats")
@rate_limit
def get_distribution_stats(date: str = None) -> str:
    """
    Get statistical distribution of firm characteristics at a specific date.
    
    Args:
        date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
    Returns:
        Distribution statistics for all characteristics
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        if date is None:
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(datadate) FROM firms")
            date = cursor.fetchone()[0]
        
        query = f"""
        SELECT 
            COUNT(*) as n_firms,
            AVG(mve) as avg_mve, MIN(mve) as min_mve, MAX(mve) as max_mve,
            AVG(bm) as avg_bm, MIN(bm) as min_bm, MAX(bm) as max_bm,
            AVG(mom12m) as avg_mom12m, MIN(mom12m) as min_mom12m, MAX(mom12m) as max_mom12m
        FROM firms
        WHERE datadate = '{date}'
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No data found for date: {date}"
        
        result = f"Distribution statistics at {date}:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error getting distribution: {str(e)}"

@tool("get_extreme_firms")
@rate_limit
def get_extreme_firms(characteristic: str, threshold: float, date: str = None) -> str:
    """
    Find firms with extreme values for a given characteristic.
    
    Args:
        characteristic: Column name (e.g., 'bm', 'mom12m', 'mve')
        threshold: Z-score threshold (e.g., 1.5 for values > 1.5 or < -1.5)
        date: Date in format 'YYYY-MM-DD' (if None, uses latest date)
    
    Returns:
        Firms with extreme values
    """
    try:
        conn = sqlite3.connect('firm_data.db')
        
        if date is None:
            cursor = conn.cursor()
            cursor.execute("SELECT MAX(datadate) FROM firms")
            date = cursor.fetchone()[0]
        
        query = f"""
        SELECT ticker, datadate, mve, bm, mom12m
        FROM firms
        WHERE datadate = '{date}' 
        AND (ABS({characteristic}) > {threshold})
        ORDER BY ABS({characteristic}) DESC
        """
        
        df = pd.read_sql_query(query, conn)
        conn.close()
        
        if df.empty:
            return f"No firms found with |{characteristic}| > {threshold} at {date}"
        
        result = f"Firms with extreme {characteristic} (|z-score| > {threshold}) at {date}:\n\n"
        result += df.to_string(index=False)
        
        return result
    except Exception as e:
        return f"Error finding extreme firms: {str(e)}"

# ============================================
# Configure LLM
# ============================================
def get_rate_limited_llm():
    """Get LLM provider with rate limiting configured."""
    llm = AG.get_llm_provider()
    try:
        llm.max_retries = 5
        llm.timeout = 60
    except:
        pass
    return llm

# ============================================
# Strategy Development Agent (Single Agent)
# ============================================
strategy_agent = Agent(
    role="Quantitative Strategy Developer",
    goal="Develop systematic BUY/HOLD/SELL rules based on firm characteristics that can be applied to all S&P 500 firms",
    backstory="""
    You are an expert quantitative strategist who creates systematic, rule-based trading strategies.
    
    CRITICAL DATA UNDERSTANDING:
    - 'mve' = log(market value of equity), represents log firm size
    - 'bm' = book-to-market ratio (value factor)
    - 'mom12m' = 12-month momentum
    - ALL features are standardized: mean = 0, standard deviation = 1
    - Values are z-scores showing standard deviations from mean
    
    Your task is to develop EXPLICIT, SYSTEMATIC RULES for generating trading signals:
    
    1. EXPLORE THE DATA (December 2019):
       - Understand the distribution of mve, bm, and mom12m
       - Identify what constitutes "extreme" values
       - Look for natural clustering or breakpoints in the data
       - Consider correlations between characteristics
    
    2. DEVELOP CLEAR RULES based on economic intuition:
       - Keep in mind the market conditions at this date - this might influence the rules you choose.
       - The following are example questions you can consider, BUT THEY ARE NOT EXHAUSTIVE: 
           - Value stocks: Should low bm (cheap) be BUY or SELL?
           - Momentum: Should high mom12m (strong performance) be BUY or SELL?
           - Size: Should mve matter for the strategy?
           - Combinations: What about value + momentum together? What about size + momentum? What about value + size? What about all three together?
    
    3. DEFINE SPECIFIC THRESHOLDS:
       Your output must include exact rules like:
       - "BUY if: bm < -0.7 AND mom12m > 0.8 AND mve > 0.2" or "BUY if: (bm > 0.5 AND mom12m < 0.8) OR mve > -0.8" or "BUY if: bm > 0.6 OR mve < 0.9"
       - "SELL if: bm > 1.25 OR mom12m < -0.9" or "SELL if: (bm < -0.8 OR mve < 0) AND mom12m < -0.9" or "SELL if: (bm > 0.9 AND mom12m < -1.0) OR mve <-0.6"
       - "HOLD: all other cases"
       - However, the above is only AN EXAMPLE - so do not simply copy the format above. You are free to include/exclude as many conditions in the if statements. You are also free to make the conditions as complicated or as simple as you like.
       - Be PRECISE in your thresholds, do not choose numbers that are nice or round - you are a quantative strategy developer.
    
    4. PROVIDE RATIONALE:
       - Why these thresholds?
       - What's the economic intuition?
       - What patterns did you observe in the data?
    
    CRITICAL REQUIREMENTS:
    - Rules must be DETERMINISTIC (same inputs → same output)
    - Use ONLY z-score comparisons (>, <, AND, OR), but these may be impacted by market conditions.
    - Define thresholds for BUY, SELL, and HOLD
    - Rules should be implementable as: if (condition) then signal
    - Keep in mind that we will use the buy signals to construct a portfolio.
    
    OUTPUT FORMAT:
    ===========================================
    SYSTEMATIC TRADING RULES
    ===========================================
    
    Data Exploration Summary:
    - [Key statistics and patterns observed]
    
    BUY RULE:
    if [exact z-score conditions]:
        signal = BUY
    
    SELL RULE:
    if [exact z-score conditions]:
        signal = SELL
    
    HOLD RULE:
    else:
        signal = HOLD
    
    Rationale:
    - [Economic reasoning for BUY rule]
    - [Economic reasoning for SELL rule]
    - [Expected signal distribution]
    
    ```
    ===========================================
    
    Be precise, systematic, and data-driven. Your rules will be applied to ~500 firms.
    """,
    llm=get_rate_limited_llm(),
    memory=True,
    verbose=True,
    max_iter=15,
    max_rpm=10
)

strategy_task = Task(
    description="""
    Develop systematic BUY/HOLD/SELL rules for S&P 500 firms at December 2019.
    
    Available characteristics (all are z-scores):
    - mve: log firm size
    - bm: log book-to-market (value)
    - mom12m: 12-month momentum
    
    Your process:
    1. Get database schema and understand available data
    2. Get distribution statistics for December 2019
    3. Explore extreme values for each characteristic
    4. Look for patterns and correlations
    5. Develop systematic rules with specific z-score thresholds
    
    CRITICAL: Your output must be EXPLICIT RULES with exact thresholds that can be 
    implemented in Python/pandas. You must give PRECISE THRESHOLDS - do not give thresholds that are only nice or round numbers.
    
    Focus on:
    - Economic intuition (value, momentum, size effects)
    - Clear, implementable thresholds
    - Balance between signal strength and diversification
    - Rules that make sense for ~500 firms
    
    Output systematic rules that I can directly implement in code.
    """,
    expected_output="""
    Complete strategy document with:
    1. Data exploration summary
    2. Explicit BUY rule with z-score thresholds
    3. Explicit SELL rule with z-score thresholds  
    4. HOLD rule (default case)
    5. Economic rationale for each rule
    
    Rules must be deterministic and implementable.
    """,
    agent=strategy_agent,
    tools=[
        get_database_schema,
        query_firm_database,
        get_distribution_stats,
        get_extreme_firms
    ]
)

crew = Crew(
    agents=[strategy_agent],
    tasks=[strategy_task],
    process=Process.sequential,
    memory=False,
    verbose=True,
    max_rpm=10
)

# ============================================
# Main Execution
# ============================================
if __name__ == "__main__":
    print("=" * 70)
    print("SYSTEMATIC STRATEGY DEVELOPER")
    print("=" * 70)
    print("\nGenerates systematic BUY/HOLD/SELL rules for S&P 500 firms")
    print("Based on z-score analysis of firm characteristics")
    print("\nThe agent will:")
    print("  1. Explore your firm characteristics data")
    print("  2. Develop systematic rules with specific thresholds")
    print("  3. Output implementation-ready strategy")
    print("\nYou can then apply the rules to all firms manually.")
    print("=" * 70)
    
    input("\nPress Enter to start strategy development...")
    
    try:
        print("\n🚀 Starting strategy development...\n")
        result = crew.kickoff()
        
        print("\n" + "=" * 70)
        print("SYSTEMATIC STRATEGY OUTPUT")
        print("=" * 70)
        print(result)
        print("=" * 70)
        
        # Save to file
        with open('systematic_strategy.txt', 'w') as f:
            f.write(str(result))
        
        print("\n✓ Strategy saved to 'systematic_strategy.txt'")
        print("\nNext steps:")
        print("  1. Review the strategy rules")
        print("  2. Implement in Python/pandas using the thresholds")
        print("  3. Apply to all firms in your dataset")
        print("  4. Generate signals for backtesting")
        
    except KeyboardInterrupt:
        print("\n\n⏸ Interrupted.")
    except Exception as e:
        print(f"\n❌ Error: {e}")
        if "429" in str(e) or "RateLimitError" in str(e):
            print("\n🚨 Rate limit hit. Wait 60 seconds and try again.")

SYSTEMATIC STRATEGY DEVELOPER

Generates systematic BUY/HOLD/SELL rules for S&P 500 firms
Based on z-score analysis of firm characteristics

The agent will:
  1. Explore your firm characteristics data
  2. Develop systematic rules with specific thresholds
  3. Output implementation-ready strategy

You can then apply the rules to all firms manually.



Press Enter to start strategy development... 



🚀 Starting strategy development...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fd5b925f-2992-4044-a1d0-53bc8ebf4b31                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quantitative Strategy Developer                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Develop systematic BUY/HOLD/SELL rules for S&P 500 firms at December 2019.                                 │
│                                                                                                                 │
│      Available characteristics (all are z-scores):                                                              │
│      - mve: log firm size                                                                                       │
│      - bm: log book-to-market (value)                                                                           │
│      - mom12m: 12-month momentum                                                                                │
│                                                                                                                 │
│      Your process:                                                                                              │
│      1. Get database schema and understand available data                                                       │
│      2. Get distribution statistics for December 2019                                                           │
│      3. Explore extreme values for each characteristic                                                          │
│      4. Look for patterns and correlations                                                                      │
│      5. Develop systematic rules with specific z-score thresholds                                               │
│                                                                                                                 │
│      CRITICAL: Your output must be EXPLICIT RULES with exact thresholds that can be                             │
│      implemented in Python/pandas. You must give PRECISE THRESHOLDS - do not give thresholds that are only      │
│  nice or round numbers.                                                                                         │
│                                                                                                                 │
│      Focus on:                                                                                                  │
│      - Economic intuition (value, momentum, size effects)                                                       │
│      - Clear, implementable thresholds                                                                          │
│      - Balance between signal strength and diversification                                                      │
│      - Rules that make sense for ~500 firms                                                                     │
│                                                                                                                 │
│      Output systematic rules that I can directly implement in code.                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quantitative Strategy Developer                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ===========================================                                                                    │
│  SYSTEMATIC TRADING RULES                                                                                       │
│  ===========================================                                                                    │
│                                                                                                                 │
│  Data Exploration Summary:                                                                                      │
│  - The distributions of bm, mom12m, and mve are centered around 0, as expected since they are standardized.     │
│  - The range of values for each characteristic is relatively wide, with some firms having extreme values        │
│  (greater than 1.5 or less than -1.5).                                                                          │
│  - There is evidence of the value, momentum, and size effects in the market.                                    │
│                                                                                                                 │
│  BUY RULE:                                                                                                      │
│  if (bm > 0.75 AND mom12m > 0.65 AND mve < 0.15) OR (bm > 1.15 AND mom12m > -0.25):                             │
│      signal = BUY                                                                                               │
│                                                                                                                 │
│  SELL RULE:                                                                                                     │
│  if (bm < -0.65 AND mom12m < -0.75) OR (mve < -1.0 AND mom12m < 0.25):                                          │
│      signal = SELL                                                                                              │
│                                                                                                                 │
│  HOLD RULE:                                                                                                     │
│  else:                                                                                                          │
│      signal = HOLD                                                                                              │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - BUY Rule:                                                                                                    │
│      - The first condition `(bm > 0.75 AND mom12m > 0.65 AND mve < 0.15)` aims to identify value stocks with    │
│  positive momentum and that are of smaller size. The thresholds are chosen to select stocks that are            │
│  relatively cheap (high bm), have shown recent positive performance (high mom12m), and are not overly large     │
│  (low mve). Smaller companies are more likely to outperform the market.                                         │
│      - The second condition `(bm > 1.15 AND mom12m > -0.25)` aims to identify deeply undervalued stocks, even   │
│  if their momentum is slightly negative, but not excess

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 78663b60-ff20-4c60-be33-f279c0ba63bd                                                                     │
│  Agent: Quantitative Strategy Developer                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: fd5b925f-2992-4044-a1d0-53bc8ebf4b31                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ===========================================                                                      │
│  SYSTEMATIC TRADING RULES                                                                                       │
│  ===========================================                                                                    │
│                                                                                                                 │
│  Data Exploration Summary:                                                                                      │
│  - The distributions of bm, mom12m, and mve are centered around 0, as expected since they are standardized.     │
│  - The range of values for each characteristic is relatively wide, with some firms having extreme values        │
│  (greater than 1.5 or less than -1.5).                                                                          │
│  - There is evidence of the value, momentum, and size effects in the market.                                    │
│                                                                                                                 │
│  BUY RULE:                                                                                                      │
│  if (bm > 0.75 AND mom12m > 0.65 AND mve < 0.15) OR (bm > 1.15 AND mom12m > -0.25):                             │
│      signal = BUY                                                                                               │
│                                                                                                                 │
│  SELL RULE:                                                                                                     │
│  if (bm < -0.65 AND mom12m < -0.75) OR (mve < -1.0 AND mom12m < 0.25):                                          │
│      signal = SELL                                                                                              │
│                                                                                                                 │
│  HOLD RULE:                                                                                                     │
│  else:                                                                                                          │
│      signal = HOLD                                                                                              │
│                                                                                                                 │
│  Rationale:                                                                                                     │
│  - BUY Rule:                                                                                                    │
│      - The first condition `(bm > 0.75 AND mom12m > 0.65 AND mve < 0.15)` aims to identify value stocks with    │
│  positive momentum and that are of smaller size. The thresholds are chosen to select stocks that are            │
│  relatively cheap (high bm), have shown recent positive performance (high mom12m), and are not overly large     │
│  (low mve). Smaller companies are more likely to outperform the market.                                         │
│      - The second condition `(bm > 1.15 AND mom12m > -


SYSTEMATIC STRATEGY OUTPUT
SYSTEMATIC TRADING RULES

Data Exploration Summary:
- The distributions of bm, mom12m, and mve are centered around 0, as expected since they are standardized.
- The range of values for each characteristic is relatively wide, with some firms having extreme values (greater than 1.5 or less than -1.5).
- There is evidence of the value, momentum, and size effects in the market.

BUY RULE:
if (bm > 0.75 AND mom12m > 0.65 AND mve < 0.15) OR (bm > 1.15 AND mom12m > -0.25):
    signal = BUY

SELL RULE:
if (bm < -0.65 AND mom12m < -0.75) OR (mve < -1.0 AND mom12m < 0.25):
    signal = SELL

HOLD RULE:
else:
    signal = HOLD

Rationale:
- BUY Rule:
    - The first condition `(bm > 0.75 AND mom12m > 0.65 AND mve < 0.15)` aims to identify value stocks with positive momentum and that are of smaller size. The thresholds are chosen to select stocks that are relatively cheap (high bm), have shown recent positive performance (high mom12m), and are not overly large (low mve)